In [3]:
! pip install mlxtend

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from mlxtend.feature_selection import SequentialFeatureSelector as SFS


In [5]:
df_train=pd.read_csv("house_price_train.csv")
df_train.drop("Id",axis=1,inplace=True) ## reduntatnt
output_col=df_train['SalePrice']
df_train=df_train.drop('SalePrice',axis=1)

In [6]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [7]:
df_train.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object

In [8]:
def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)

In [9]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [10]:
df_train.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF'

In [11]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


In [12]:
def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int64" or df_train[cols].dtype=="float64":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)


In [13]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [14]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [15]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [16]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [1]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(20, 20)

# for ax, col in zip(axes, continuous_cols):
#     print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [18]:
# plt.figure(figsize=(20,15))
# sns.heatmap(df_train.loc[:,continuous_cols].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

We are encoding categorical columns using label encoder as 0,1,2

In [20]:

def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train

We have encoded our categorical columns using labelencoder. The reason we haven't used onehotencoder is explained below:
Suppose the categorical column "LotShape" have 5 categories then onehotencoder will generate 5 new columns . Now when we know Sequential Feature Selector(SFS) selects some column which gives us highest accuracy, now if SFS selects 4 columns(categories) of one encoded "LotShape" then the training accuracy might give us good results but when test or when we have to predict new datapoint then our accuracy will be vey low. 

In [21]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="labelencoder")

In [22]:
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,1,0,0,0,1,0,0,0,0,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,1,0,1,0,0,0,0,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,1,0,1,0,1,0,0,0,0,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,1,0,0,1,0,0,0,0,0,2
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,1,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,0,1,0,1,0,0,0,0,0
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,0,2,1,0,0,0,0,0,0,0
1457,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,1,0,0,0,1,0,0,0,0,0
1458,20,68.0,9717,5,6,1950,1996,0.0,49,1029,...,1,0,0,0,0,0,0,0,0,0


In [23]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>0:
        print(col)

Using Min Max scaler to make values in range 0 to 1.

In [25]:
scaler=scaler=MinMaxScaler()
df_train=pd.DataFrame(scaler.fit_transform(df_train),columns=df_train.columns,)
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,0.235294,0.150685,0.033420,0.666667,0.500,0.949275,0.883333,0.122500,0.125089,0.000000,...,0.333333,0.000000,0.00,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.202055,0.038795,0.555556,0.875,0.753623,0.433333,0.000000,0.173281,0.000000,...,0.000000,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
2,0.235294,0.160959,0.046507,0.666667,0.500,0.934783,0.866667,0.101250,0.086109,0.000000,...,0.333333,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
3,0.294118,0.133562,0.038561,0.666667,0.500,0.311594,0.333333,0.000000,0.038271,0.000000,...,0.333333,0.000000,0.00,0.2,0.0,0.0,0.0,0.0,0.0,0.4
4,0.235294,0.215753,0.060576,0.777778,0.500,0.927536,0.833333,0.218750,0.116052,0.000000,...,0.333333,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.235294,0.140411,0.030929,0.555556,0.500,0.920290,0.833333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1456,0.000000,0.219178,0.055505,0.555556,0.625,0.768116,0.633333,0.074375,0.139972,0.110583,...,0.000000,0.333333,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1457,0.294118,0.154110,0.036187,0.666667,1.000,0.500000,0.933333,0.000000,0.048724,0.000000,...,0.333333,0.000000,0.00,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1458,0.000000,0.160959,0.039342,0.444444,0.625,0.565217,0.766667,0.000000,0.008682,0.698100,...,0.333333,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
from sklearn.metrics import r2_score

In [27]:
X=df_train
Y=output_col

In [34]:
final_accuracies={}
algorithms_tested=["LinearRegression()","RandomForestRegressor()","SVR()","KNeighborsRegressor()","SGDRegressor()","DecisionTreeRegressor()","ElasticNet()","Lasso()","Ridge()"]
for al in algorithms_tested:
    final_accuracies[al]=0

In [35]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=None,shuffle=True)


Here we are finally using Sequential Feature Selector from mlxtend library.It sequentially takes features, one by one and adds into our final model so to check which feature set gives us best score. It continuosuly checks r2_score whenever a feature is added. This is done so that we can get a subset of features which give us highest r2_score and remove the unwanted or noisy columns.


The minimum features we have selected as 3 . We have also set the floating parameter as True since the add an additional exclusion or inclusion step to remove features once they were included (or excluded) so that a larger number of feature subset combinations can be sampled.  

In [36]:
def sf_selector(algo:str,forward:bool,floating:bool,scoring:int,X_train,Y_train,X_test,Y_test,min_featues:int,cv:int,verbose:int):
    model=eval(algo)
    sfs=SFS(model, 
           k_features=(min_featues,len(X_train.columns)), 
           forward=forward, 
           floating=floating, 
           verbose=verbose,
           scoring=scoring,
           cv=cv,
           n_jobs=-1)
    sfs=sfs.fit(X_train, Y_train)
    print("Selected Features",sfs.k_feature_names_)
    model_=eval(algo)
    model_.fit(X_train.loc[:,list(sfs.k_feature_names_)],Y_train)
    score=model_.score(X_test.loc[:,list(sfs.k_feature_names_)],Y_test)
    
    print(f"Prediction using selected features using {model_} is :",score)
    # if final_accuracies[algo]<score:
        
    final_accuracies[algo]=score


In [37]:
sf_selector(algo="LinearRegression()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:    0.1s finished

[2023-01-31 17:39:56] Features: 1/75 -- score: 0.6267792243690402[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  74 out of  74 | elapsed:    0.1s finished

[2023-01-31 17:39:56] Features: 2/75 -- score: 0.6989758093398494[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  73 out of  73 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2023-01-31 17:39:57] Features: 3/75 -- score: 0.7254197703137283[Parallel(n_jobs=-1)]: Using

Selected Features ('MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'MasVnrArea', 'GrLivArea', 'BsmtFullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'WoodDeckSF', '3SsnPorch', 'ScreenPorch', 'MSZoning', 'Street', 'Neighborhood', 'Condition1', 'BldgType', 'RoofStyle', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition')
Prediction using selected features using LinearRegression() is : 0.8154178700144578


[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  74 out of  74 | elapsed:    0.4s finished

[2023-01-31 17:40:51] Features: 75/75 -- score: -1492338529.421611

In [ ]:
sf_selector(algo="RandomForestRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="SVR()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="KNeighborsRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="DecisionTreeRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="SGDRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="ElasticNet()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="Lasso()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="Ridge()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [88]:

# sfs1 = SFS(LinearRegression(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=0,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


sfs1 max score features are : ('MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'MasVnrArea', 'GrLivArea', 'BsmtFullBath', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'YrSold', 'MSZoning', 'Street', 'LandContour', 'Utilities', 'Neighborhood', 'Condition1', 'HouseStyle', 'RoofStyle', 'MasVnrType', 'ExterQual', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType')
sfs1 max score is : 0.7969288395420955



In [89]:
# model_=LinearRegression()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["LinearRegression"]=score

Prediction using selected features is : 0.8832585762908746


In [3]:

# sfs1 = SFS(RandomForestRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [91]:
# model_=RandomForestRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["RandomForestRegressor"]=score

Prediction using selected features is : 0.9066906695910057


In [4]:
# sfs1 = SFS(SVR(), 
#            k_features=(3,len(X_train.columns)), 
#            forward=True, 
#            floating=False, 
#            verbose=2,
#            scoring='r2',
#            cv=5,
#            n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [5]:
# model_=SVR()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["SVR"]=score

In [6]:
# sfs1 = SFS(KNeighborsRegressor(), 
#            k_features=(3,len(X_train.columns)), 
#            forward=True, 
#            floating=False, 
#            verbose=2,
#            scoring='r2',
#            cv=5,
#            n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [7]:
# model_=KNeighborsRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["KNeighborsRegressor"]=score

In [8]:
# sfs1 = SFS(DecisionTreeRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=2,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [9]:
# model_=DecisionTreeRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["DecisionTreeRegressor"]=score

In [10]:
# sfs1 = SFS(SGDRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=2,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [11]:
# model_=SGDRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["SGDRegressor"]=score

In [12]:
# sfs1 = SFS(ElasticNet(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=2,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [13]:
# model_=ElasticNet()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["ElasticNet"]=score

In [14]:
# sfs1 = SFS(Lasso(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=2,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [15]:
# model_=Lasso()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["Lasso"]=score

In [16]:
# sfs1 = SFS(Ridge(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=2,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [17]:
# model_=Ridge()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["Ridge"]=score

In [106]:
accuracies

{'LinearRegression': 0.8832585762908746,
 'RandomForestRegressor': 0.9066906695910057,
 'SVR': 0.848995271814013,
 'KNeighborsRegressor': 0.8539298072616761,
 'DecisionTreeRegressor': 0.7761212917265028,
 'SGDRegressor': 0.812159923737776,
 'ElasticNet': -0.00037849311354176685,
 'Lasso': -0.00037849311354176685,
 'Ridge': 0.8817114235566021}